In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from ast import literal_eval
import matplotlib.pyplot as plt

In [2]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [58]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

# cust_data['article_id'] = cust_data.apply(lambda x :  'a'+ str(x.article_id), axis = 1)

def str_article(x):
    return 'a'+str(x)

cust_data['article_id'] = np.vectorize(str_article)(cust_data['article_id'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [59]:
cust_data['article_max_purchase_date'] = cust_data.groupby(['article_id'])['t_dat_datetime'].transform('max')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
cust_data['product_purchase'] = cust_data.groupby(['article_id'])['customer_id'].transform('nunique')

cust_data = cust_data[cust_data['product_purchase'] >= 12]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
cust_data['customer_visits'] = cust_data.groupby(['customer_id'])['t_dat'].transform('nunique')

cust_data = cust_data[cust_data['customer_visits'] >= 12]

In [62]:
cust_data['customer_prod_count'] = cust_data.groupby(['customer_id'])['article_id'].transform('nunique')

cust_data = cust_data[cust_data['customer_prod_count'] >= 4]

In [63]:
cust_data['product_max_purc_date'] = cust_data.groupby(['article_id'])['t_dat_datetime'].transform('max')

In [64]:
cust_data_2020_9 = cust_data[(cust_data['product_max_purc_date'].dt.year==2020)&
                                     (cust_data['product_max_purc_date'].dt.month == 9)]

In [65]:
cust_data_2020_9['product_purchase'] = cust_data_2020_9.groupby(['article_id'])['customer_id'].transform('nunique')

cust_data_2020_9 = cust_data_2020_9[cust_data_2020_9['product_purchase'] >= 12]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
cust_data_2020_9 = cust_data_2020_9.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)
      ).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index().\
assign(prods = lambda x:  list(x['prods']))


# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 

In [67]:
cust_data_2020_9['prods_len'] = cust_data_2020_9.apply(lambda x: len(x.prods) , axis =1)


In [68]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

article_df['article_id'] = 'a'+article_df['article_id'].astype('str')

In [69]:
cust_data = pd.merge(cust_data,
                     cust_data_2020_9['customer_id'],
                     left_on = 'customer_id',
                     right_on = 'customer_id',
                    )

In [70]:
cust_data = pd.merge(pd.merge(cust_data ,
         article_df[['article_id','product_group_name']],
         left_on = 'article_id',
         right_on = 'article_id'
        ),
         customer_df[['customer_id','age']],
         left_on = 'customer_id',
         right_on = 'customer_id'
        )

In [71]:
cust_features = cust_data.groupby(['customer_id','age']).agg( tran_count = ('t_dat','nunique'),
                                       article_count = ('article_id','nunique'),
                                       spend = ('price','sum')
                                      ).reset_index()

In [72]:
cust_features_1 = pd.pivot_table(cust_data[['customer_id','product_group_name','t_dat']],
         index = 'customer_id',
         columns = 'product_group_name',
         values = 't_dat',
         aggfunc = 'nunique'
        ).fillna(0).reset_index()

In [73]:
cust_features = pd.merge(cust_features, 
                         cust_features_1,
                         left_on = 'customer_id',
                         right_on = 'customer_id',
                        )

In [74]:
# cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 
# cust_data_2020_9["prods"].values[:5]

In [75]:
article_df['index_group_name'].unique()

array(['Ladieswear', 'Baby/Children', 'Menswear', 'Sport', 'Divided'],
      dtype=object)

In [76]:
cust_features.groupby(['age']).agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending = False).head(60)

,age,customer_id
9,25.0,4994
8,24.0,4948
10,26.0,4718
7,23.0,4598
11,27.0,4294
6,22.0,3915
5,21.0,3705
12,28.0,3646
13,29.0,3279
14,30.0,3078


In [77]:
def age_groups(x):
    if x <= 10:
        res = 'kids'
    elif x > 10 and x <= 20:
        res ='teen'
    elif x > 20 and x <= 23:
        res = 'early_20_1'
    elif x > 23 and x <= 25:
        res = 'early_20_2'
    elif x > 25 and  x<= 27:
        res = 'late_20_1'
    elif x > 27 and  x<= 30:
        res = 'late_20_2'
    elif x > 30 and x <= 35:
        res = 'early_30'
    elif x > 35 and x <= 40:
        res = 'late_30'
    elif x > 40 and x <= 45:
        res = 'early_40'
    elif x > 45 and x <= 50:
        res = 'late_40'
    elif x > 50 and x <= 55:
        res = 'early_50'
    elif x > 55 and x <= 60:
        res = 'late_50'
    elif x > 60:
        res = 'above_60'
    return res
        
cust_features['age_bracket'] = np.vectorize(age_groups)(cust_features.age)

In [78]:
cust_features.groupby(['age_bracket']).agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending = False).head(60)

,age_bracket,customer_id
1,early_20_1,12218
7,late_20_2,10003
2,early_20_2,9942
5,early_50,9518
9,late_40,9339
3,early_30,9208
6,late_20_1,9012
4,early_40,5665
10,late_50,5599
8,late_30,4782


In [79]:
model_data = pd.merge(cust_data_2020_9,
                      cust_features.query('age_bracket == "teen"'),
                      left_on = 'customer_id',
                      right_on = 'customer_id'
                     )

In [80]:
# import gc

# del cust_data_2020_9
# gc.collect()

# del cust_features
# gc.collect()

In [81]:
# import gc
# # del customer_df
# # gc.collect()

# del article_df
# gc.collect()


In [82]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [83]:
genre_mlb = mlb.fit_transform(model_data['prods'])
genre_mlb

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [84]:
len(genre_mlb[0:1][0])

8415

In [85]:
genre = mlb.inverse_transform(genre_mlb[0:1])
genre

[('a316085001',
  'a372860001',
  'a399223034',
  'a399223059',
  'a688537004',
  'a732842002',
  'a748269009',
  'a777273001',
  'a778064001',
  'a817354001',
  'a817491006',
  'a841434007')]

In [86]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

In [87]:
model_data.shape[0]*0.8

3545.6000000000004

In [88]:
X_train = model_data.drop('age_bracket', axis = 1).iloc[0:3500,2:]
y_train = genre_mlb[0:3500]
X_test = model_data.drop('age_bracket', axis = 1).iloc[3500:,2:]
y_test = genre_mlb[3500:]

In [89]:
model = Sequential()
model.add(Dense(5000, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(4000, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(600, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

In [90]:
# def macro_soft_f1(y, y_hat):
#     """Compute the macro soft F1-score as a cost.
#     Average (1 - soft-F1) across all labels.
#     Use probability values instead of binary predictions.
    
#     Args:
#         y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix of shape (BATCH_SIZE, N_LABELS)
        
#     Returns:
#         cost (scalar Tensor): value of the cost function for the batch
#     """
    
#     y = tf.cast(y, tf.float32)
#     y_hat = tf.cast(y_hat, tf.float32)
#     tp = tf.reduce_sum(y_hat * y, axis=0)
#     fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
#     fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
#     soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
#     macro_cost = tf.reduce_mean(cost) # average on all labels
    
#     return macro_cost

In [91]:
# def macro_f1(y, y_hat, thresh=0.7):
#     """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
#     Args:
#         y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
#         thresh: probability value above which we predict positive
        
#     Returns:
#         macro_f1 (scalar Tensor): value of macro F1 for the batch
#     """
#     y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
#     tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
#     fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
#     fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
#     f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     macro_f1 = tf.reduce_mean(f1)
#     return macro_f1

In [92]:
# correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(pred)), tf.round(y_test))

# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [93]:
# all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32),1)
# accuracy2 = tf.reduce_mean(all_labels_true)

In [94]:
# def accuracy2(y, y_hat):
    
#     correct_prediction = tf.equal(tf.round(tf.nn.sigmoid(y_hat)), tf.round(y))
   
#     all_labels_true = tf.reduce_min(tf.cast(correct_prediction, tf.float32),1)
    
#     accuracy2 = tf.reduce_mean(all_labels_true)
    
#     return accuracy2

In [95]:
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss=macro_soft_f1,
#   metrics=[macro_f1], optimizer=sgd)

model.compile(
    loss="binary_crossentropy", optimizer="sgd", metrics=['categorical_accuracy']
)

model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
350/350 [==============================] - 16s 45ms/step - loss: 0.6806 - categorical_accuracy: 0.0000e+00
Epoch 2/20
350/350 [==============================] - 16s 46ms/step - loss: 0.5911 - categorical_accuracy: 0.0000e+00
Epoch 3/20
350/350 [==============================] - 16s 45ms/step - loss: 0.2481 - categorical_accuracy: 2.8571e-04
Epoch 4/20
350/350 [==============================] - 16s 45ms/step - loss: 0.0463 - categorical_accuracy: 0.0000e+00
Epoch 5/20
350/350 [==============================] - 15s 44ms/step - loss: 0.0237 - categorical_accuracy: 0.0014
Epoch 6/20
350/350 [==============================] - 16s 47ms/step - loss: 0.0189 - categorical_accuracy: 0.0031
Epoch 7/20
350/350 [==============================] - 15s 44ms/step - loss: 0.0170 - categorical_accuracy: 0.0023
Epoch 8/20
350/350 [==============================] - 16s 46ms/step - loss: 0.0161 - categorical_accuracy: 0.0049
Epoch 9/20
350/350 [==============================] - 16s 45ms/step - lo

In [103]:
preds = model.predict(X_test)
# preds[preds>=0.7] = 1
# preds[preds<0.7] = 0
# score = compare preds and y_test

In [48]:
preds

NameError: name 'preds' is not defined

In [ ]:
len(preds[0])

In [97]:
score = model.evaluate(X_test, y_test, batch_size=100)
score

52/52 [==============================] - 2s 33ms/step - loss: 0.0056 - categorical_accuracy: 0.0252


[0.005645365454256535, 0.025193799287080765]

In [98]:
genre_mlb[0:1].shape

(1, 14217)

In [99]:
preds[0:1].astype(int).shape

(1, 14217)

In [100]:
mlb.inverse_transform(genre_mlb[0:1])

[('a529589002',
  'a731851001',
  'a733098018',
  'a739144004',
  'a745232001',
  'a761285002',
  'a779664001',
  'a806529003',
  'a808841001',
  'a817472002',
  'a820671001',
  'a855834001')]

In [101]:
mlb.inverse_transform(preds[0:1].astype(int))

[()]

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)